In [10]:
import pandas as pd

In [11]:
import os
from dotenv import load_dotenv

load_dotenv()

True

In [12]:
file_path = "./output/rag_data.jsonl"

df = pd.read_json(
        path_or_buf=file_path, 
        lines=True, 
        encoding='utf-8' # 파일 인코딩에 맞게 설정 (대부분 'utf-8')
    )
df

,qid,imdb_id,tmdb_id,type,title_ko,title_en,year,overview,genres,poster_path,...,runtime_min,cast,directors,writers,ott_streaming_kr,ott_rent_kr,ott_buy_kr,keywords,rating_kr,rag_text
0,http://www.wikidata.org/entity/Q110393291,tt13498824,760497,movie,승부,The Match,2025,세계 최고 바둑 대회에서 국내 최초 우승자가 된 조훈현. 전 국민적 영웅으로 대접받...,[드라마],/x1xHVv5rdvvCjF5wmGhtbHrlUgo.jpg,...,115.0,"[이병헌, 유아인, 고창석, 현봉식, 문정희, 조우진, 손종학, 김강훈, 남문철, ...",[김형주],"[김형주, 윤종빈]","[Netflix, Netflix Standard with Ads]",[],[],"[based on true story, go]",12,[제목] 승부\n[영문 제목] The Match\n[줄거리] 세계 최고 바둑 대회에...
1,http://www.wikidata.org/entity/Q110585119,tt14044924,793058,movie,하이파이브,Hi-Five,2025,"태권소녀 완서, 작가 지망생 지성, 후레쉬 매니저 선녀, FM 작업반장 약선 그리...","[SF, 액션, 코미디, 판타지]",/xpqa2ShXecFxMUGKv82Zx56ZoUD.jpg,...,120.0,"[이재인, 안재홍, 유아인, 라미란, 김희원, 오정세, 신구, 진영, 진희경, 장광]",[강형철],[강형철],[Disney Plus],[],[],"[super power, organ transplant, superhero team]",15,"[제목] 하이파이브\n[영문 제목] Hi-Five\n[줄거리] 태권소녀 완서, 작가..."
2,http://www.wikidata.org/entity/Q112977213,tt12299608,696506,movie,미키 17,Mickey 17,2025,친구 티모와 함께 차린 마카롱 가게가 쫄딱 망해 거액의 빚을 지고 못 갚으면 죽이겠...,"[SF, 코미디, 모험]",/mH7QnJDxQibVZw0M66IBZbsw2O6.jpg,...,137.0,"[로버트 패틴슨, 나오미 애키, 스티븐 연, 마크 러팔로, 토니 콜렛, 아나마리아 ...",[봉준호],[봉준호],[],[Google Play Movies],[Google Play Movies],"[based on novel or book, dark comedy, space tr...",15,[제목] 미키 17\n[영문 제목] Mickey 17\n[줄거리] 친구 티모와 함께...
3,http://www.wikidata.org/entity/Q120922062,tt28488187,973628,movie,보통의 가족,A Normal Family,2025,물질적 욕망을 우선시하며 살인자의 변호도 마다하지 않는 변호사 재완과 원리원칙을 중...,"[드라마, 스릴러]",/chuNOF8p6Y1mJ0McFKa17ojXAsL.jpg,...,109.0,"[설경구, 장동건, 김희애, 수현, 홍예지, 김정철, 최리, 유수빈, 변중희, Ah...",[허진호],"[Park Eun-kyo, 박준석]",[Disney Plus],"[wavve, Google Play Movies]","[wavve, Google Play Movies]","[based on novel or book, dark comedy]",12,[제목] 보통의 가족\n[영문 제목] A Normal Family\n[줄거리] 물질...
4,http://www.wikidata.org/entity/Q125971387,tt12300742,701387,movie,부고니아,None,2025,"벌들은 사라지고, 지구는 병들고 있고, 인류는 고통받고 있다. 거대 바이오 기업의 ...","[SF, 범죄, 스릴러]",/hHCuWbwZvywOTWTk1CVIRMQgnkB.jpg,...,119.0,"[엠마 스톤, 제시 플레먼스, 에이단 델비스, 스타브로스 할키어스, 알리시아 실버스...",[요르고스 란티모스],[윌 트레이시],[],[],[],"[saving the world, capitalism, kidnapping, par...",15,"[제목] 부고니아\n[영문 제목] nan\n[줄거리] 벌들은 사라지고, 지구는 병들..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
855,http://www.wikidata.org/entity/Q50322115,tt4722674,383353,movie,Naneun Seonmuda,I Am Sun Mu,2015,서울에서 김정일 체제에 대한 풍자 그림을 그리고 자신의 탈북으로 비롯된 아픔을 극복...,"[애니메이션, 다큐멘터리]",NaN,...,NaN,"[Sun Mu, Angeline, Cui Xianji, Liang Kegang, H...",[Adam Sjöberg],[Adam Sjöberg],[],NaN,NaN,"[biography, news, north korean defector, north...",,[제목] Naneun Seonmuda\n[영문 제목] I Am Sun Mu\n[줄거...
856,http://www.wikidata.org/entity/Q55734574,tt5258552,449802,movie,중급불어,None,2015,,[드라마],NaN,...,NaN,"[Yoon Geum-sun, Gregor Mc Ghee]",[Yann Kerloc'h],[Yann Kerloc'h],[],NaN,NaN,"[exam, viddsee, short film]",,[제목] 중급불어\n[영문 제목] nan\n[추가 요약] 《중급불어》는 2015년에...
857,http://www.wikidata.org/entity/Q65241097,tt7520904,601068,movie,다방의 푸른 꿈,Try To Remember,2015,전쟁의 폐허 속에서 음악에 대한 천부적인 재능을 통해 국내 최초 걸그룹 결성! 듣는...,"[다큐멘터리, 음악]",NaN,...,NaN,"[김민자, 김숙자, 김애자, Nan-yeong Lee, Kim Hae-song]",[Kim Dae-hyun],[Kim Dae-hyun],[],NaN,NaN,"[girl group, k-pop]",,[제목] 다방의 푸른 꿈\n[영문 제목] Try To Remember\n[줄거리] ...
858,http://www.wikidata.org/entity/Q65300188,tt5358748,510589,movie,퀴어영화 나비: 어른들의 일,Butterfly: The Adult World,2015,,[드라마],NaN,...,NaN,"[Daeyoung Kim, Yeongdeok Lim, Gwangmu Moon, Se...",[Inkyu Baek],[Inkyu Baek],"[wavve, TVING]",NaN,NaN,"[gay club, boys' love (bl)]",,[제목] 퀴어영화 나비: 어른들의 일\n[영문 제목] Butterfly: The A...


In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 860 entries, 0 to 859
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   qid               860 non-null    object 
 1   imdb_id           860 non-null    object 
 2   tmdb_id           860 non-null    int64  
 3   type              860 non-null    object 
 4   title_ko          860 non-null    object 
 5   title_en          849 non-null    object 
 6   year              860 non-null    int64  
 7   overview          860 non-null    object 
 8   genres            860 non-null    object 
 9   poster_path       537 non-null    object 
 10  backdrop_path     530 non-null    object 
 11  runtime_min       540 non-null    float64
 12  cast              860 non-null    object 
 13  directors         860 non-null    object 
 14  writers           860 non-null    object 
 15  ott_streaming_kr  860 non-null    object 
 16  ott_rent_kr       540 non-null    object 
 1

In [14]:
columns_to_keep = [
    'title_ko', 
    'year', 
    'genres',
    'cast', 
    'directors', 
    'ott_streaming_kr', 
    'rag_text',
    'runtime_min',
    'rating_kr'
]

df = df[columns_to_keep] 

# 결과 확인
print("✅ 선택된 열만 남은 새로운 DataFrame의 정보:")
print(df.info())
print("\n첫 5개 행:")
print(df.head())

✅ 선택된 열만 남은 새로운 DataFrame의 정보:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 860 entries, 0 to 859
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   title_ko          860 non-null    object 
 1   year              860 non-null    int64  
 2   genres            860 non-null    object 
 3   cast              860 non-null    object 
 4   directors         860 non-null    object 
 5   ott_streaming_kr  860 non-null    object 
 6   rag_text          860 non-null    object 
 7   runtime_min       540 non-null    float64
 8   rating_kr         860 non-null    object 
dtypes: float64(1), int64(1), object(7)
memory usage: 60.6+ KB
None

첫 5개 행:
  title_ko  year              genres  \
0       승부  2025               [드라마]   
1    하이파이브  2025  [SF, 액션, 코미디, 판타지]   
2    미키 17  2025       [SF, 코미디, 모험]   
3   보통의 가족  2025          [드라마, 스릴러]   
4     부고니아  2025       [SF, 범죄, 스릴러]   

                              

In [15]:
df = df.fillna({
        'runtime_min': ''
    })

df

,title_ko,year,genres,cast,directors,ott_streaming_kr,rag_text,runtime_min,rating_kr
0,승부,2025,[드라마],"[이병헌, 유아인, 고창석, 현봉식, 문정희, 조우진, 손종학, 김강훈, 남문철, ...",[김형주],"[Netflix, Netflix Standard with Ads]",[제목] 승부\n[영문 제목] The Match\n[줄거리] 세계 최고 바둑 대회에...,115.0,12
1,하이파이브,2025,"[SF, 액션, 코미디, 판타지]","[이재인, 안재홍, 유아인, 라미란, 김희원, 오정세, 신구, 진영, 진희경, 장광]",[강형철],[Disney Plus],"[제목] 하이파이브\n[영문 제목] Hi-Five\n[줄거리] 태권소녀 완서, 작가...",120.0,15
2,미키 17,2025,"[SF, 코미디, 모험]","[로버트 패틴슨, 나오미 애키, 스티븐 연, 마크 러팔로, 토니 콜렛, 아나마리아 ...",[봉준호],[],[제목] 미키 17\n[영문 제목] Mickey 17\n[줄거리] 친구 티모와 함께...,137.0,15
3,보통의 가족,2025,"[드라마, 스릴러]","[설경구, 장동건, 김희애, 수현, 홍예지, 김정철, 최리, 유수빈, 변중희, Ah...",[허진호],[Disney Plus],[제목] 보통의 가족\n[영문 제목] A Normal Family\n[줄거리] 물질...,109.0,12
4,부고니아,2025,"[SF, 범죄, 스릴러]","[엠마 스톤, 제시 플레먼스, 에이단 델비스, 스타브로스 할키어스, 알리시아 실버스...",[요르고스 란티모스],[],"[제목] 부고니아\n[영문 제목] nan\n[줄거리] 벌들은 사라지고, 지구는 병들...",119.0,15
...,...,...,...,...,...,...,...,...,...
855,Naneun Seonmuda,2015,"[애니메이션, 다큐멘터리]","[Sun Mu, Angeline, Cui Xianji, Liang Kegang, H...",[Adam Sjöberg],[],[제목] Naneun Seonmuda\n[영문 제목] I Am Sun Mu\n[줄거...,,
856,중급불어,2015,[드라마],"[Yoon Geum-sun, Gregor Mc Ghee]",[Yann Kerloc'h],[],[제목] 중급불어\n[영문 제목] nan\n[추가 요약] 《중급불어》는 2015년에...,,
857,다방의 푸른 꿈,2015,"[다큐멘터리, 음악]","[김민자, 김숙자, 김애자, Nan-yeong Lee, Kim Hae-song]",[Kim Dae-hyun],[],[제목] 다방의 푸른 꿈\n[영문 제목] Try To Remember\n[줄거리] ...,,
858,퀴어영화 나비: 어른들의 일,2015,[드라마],"[Daeyoung Kim, Yeongdeok Lim, Gwangmu Moon, Se...",[Inkyu Baek],"[wavve, TVING]",[제목] 퀴어영화 나비: 어른들의 일\n[영문 제목] Butterfly: The A...,,


In [16]:
from sklearn.preprocessing import MultiLabelBinarizer

print("--- 1. 원본 DataFrame 정보 ---")
print(df.info())
print("\n--- 원본 'ott_streaming_kr' 열의 첫 번째 값 (리스트 타입 확인) ---")
print(df['ott_streaming_kr'].iloc[0])
print("\n--- 원본 'genres' 열의 첫 번째 값 (리스트 타입 확인) ---")
print(df['genres'].iloc[0])

--- 1. 원본 DataFrame 정보 ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 860 entries, 0 to 859
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   title_ko          860 non-null    object
 1   year              860 non-null    int64 
 2   genres            860 non-null    object
 3   cast              860 non-null    object
 4   directors         860 non-null    object
 5   ott_streaming_kr  860 non-null    object
 6   rag_text          860 non-null    object
 7   runtime_min       860 non-null    object
 8   rating_kr         860 non-null    object
dtypes: int64(1), object(8)
memory usage: 60.6+ KB
None

--- 원본 'ott_streaming_kr' 열의 첫 번째 값 (리스트 타입 확인) ---
['Netflix', 'Netflix Standard with Ads']

--- 원본 'genres' 열의 첫 번째 값 (리스트 타입 확인) ---
['드라마']


In [17]:
# ----------------------------------------------------
# 1. 'cast', 'directors' 리스트를 문자열로 변환 ( [ ] 제거 )
# ----------------------------------------------------
# .apply()와 lambda 함수를 사용하여 리스트의 모든 항목을 ', ' (콤마+공백)로 연결합니다.
# 만약 리스트가 비어있으면 (예: []) 자동으로 빈 문자열('')이 됩니다.
df['casts'] = df['cast'].apply(
    lambda x: ', '.join(x) if isinstance(x, list) else ''
)
df['director'] = df['directors'].apply(
    lambda x: ', '.join(x) if isinstance(x, list) else ''
)

# ----------------------------------------------------
# 2. 'ott_streaming_kr' 리스트를 One-Hot 인코딩
# ----------------------------------------------------
# Scikit-learn의 MultiLabelBinarizer가 이 작업에 가장 적합합니다.
mlb = MultiLabelBinarizer()

# .fit_transform()을 사용하여 리스트를 원핫 인코딩 배열로 변환
ott_one_hot = mlb.fit_transform(df['ott_streaming_kr'])

# 원핫 인코딩된 배열을 DataFrame으로 변환
# 열 이름은 실제 OTT 플랫폼 이름이 됩니다 (예: 'Netflix', 'Disney Plus')
# add_prefix('ott_')를 사용하여 열 이름을 'ott_Netflix' 등으로 만듭니다. (선택 사항)
ott_df = pd.DataFrame(ott_one_hot, 
                      columns=mlb.classes_, 
                      index=df.index).add_prefix('ott_')

# .fit_transform()을 사용하여 장르 리스트를 원핫 인코딩 배열로 변환
genre_one_hot = mlb.fit_transform(df['genres'])

# 원핫 인코딩된 배열을 DataFrame으로 변환
# 열 이름은 실제 장르 이름이 됩니다. add_prefix('genre_')를 사용하여 열 이름을 'genre_드라마' 등으로 만듭니다.
genre_df = pd.DataFrame(genre_one_hot, 
                        columns=mlb.classes_, 
                        index=df.index).add_prefix('genre_')

# 기존 DataFrame(df)에 새로운 One-Hot DataFrame(genre_df)을 병합
df = pd.concat([df, genre_df], axis=1)

# 기존 DataFrame(df)에 새로운 원핫 인코딩 DataFrame(ott_df)을 병합
df = pd.concat([df, ott_df], axis=1)

# ----------------------------------------------------
# 3. 정리: 원본 리스트 열들 제거
# ----------------------------------------------------
df = df.drop(columns=['cast', 'directors', 'ott_streaming_kr', 'genres'])

print("\n\n--- 3. 변환 완료된 DataFrame 정보 ---")
print(df.info())

print("\n\n--- 4. 최종 DataFrame 결과 (일부 열) ---")
print(df[['title_ko', 'casts', 'director', 'ott_Disney Plus', 'ott_Netflix']].head())



--- 3. 변환 완료된 DataFrame 정보 ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 860 entries, 0 to 859
Data columns (total 34 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   title_ko                       860 non-null    object
 1   year                           860 non-null    int64 
 2   rag_text                       860 non-null    object
 3   runtime_min                    860 non-null    object
 4   rating_kr                      860 non-null    object
 5   casts                          860 non-null    object
 6   director                       860 non-null    object
 7   genre_Action & Adventure       860 non-null    int64 
 8   genre_Reality                  860 non-null    int64 
 9   genre_SF                       860 non-null    int64 
 10  genre_Sci-Fi & Fantasy         860 non-null    int64 
 11  genre_가족                       860 non-null    int64 
 12  genre_공포                       

In [18]:
df

,title_ko,year,rag_text,runtime_min,rating_kr,casts,director,genre_Action & Adventure,genre_Reality,genre_SF,...,genre_전쟁,genre_코미디,genre_판타지,ott_Disney Plus,ott_FilmBox+,ott_Netflix,ott_Netflix Standard with Ads,ott_TVING,ott_Watcha,ott_wavve
0,승부,2025,[제목] 승부\n[영문 제목] The Match\n[줄거리] 세계 최고 바둑 대회에...,115.0,12,"이병헌, 유아인, 고창석, 현봉식, 문정희, 조우진, 손종학, 김강훈, 남문철, 주진모",김형주,0,0,0,...,0,0,0,0,0,1,1,0,0,0
1,하이파이브,2025,"[제목] 하이파이브\n[영문 제목] Hi-Five\n[줄거리] 태권소녀 완서, 작가...",120.0,15,"이재인, 안재홍, 유아인, 라미란, 김희원, 오정세, 신구, 진영, 진희경, 장광",강형철,0,0,1,...,0,1,1,1,0,0,0,0,0,0
2,미키 17,2025,[제목] 미키 17\n[영문 제목] Mickey 17\n[줄거리] 친구 티모와 함께...,137.0,15,"로버트 패틴슨, 나오미 애키, 스티븐 연, 마크 러팔로, 토니 콜렛, 아나마리아 바...",봉준호,0,0,1,...,0,1,0,0,0,0,0,0,0,0
3,보통의 가족,2025,[제목] 보통의 가족\n[영문 제목] A Normal Family\n[줄거리] 물질...,109.0,12,"설경구, 장동건, 김희애, 수현, 홍예지, 김정철, 최리, 유수빈, 변중희, Ahn...",허진호,0,0,0,...,0,0,0,1,0,0,0,0,0,0
4,부고니아,2025,"[제목] 부고니아\n[영문 제목] nan\n[줄거리] 벌들은 사라지고, 지구는 병들...",119.0,15,"엠마 스톤, 제시 플레먼스, 에이단 델비스, 스타브로스 할키어스, 알리시아 실버스톤...",요르고스 란티모스,0,0,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
855,Naneun Seonmuda,2015,[제목] Naneun Seonmuda\n[영문 제목] I Am Sun Mu\n[줄거...,,,"Sun Mu, Angeline, Cui Xianji, Liang Kegang, Ho...",Adam Sjöberg,0,0,0,...,0,0,0,0,0,0,0,0,0,0
856,중급불어,2015,[제목] 중급불어\n[영문 제목] nan\n[추가 요약] 《중급불어》는 2015년에...,,,"Yoon Geum-sun, Gregor Mc Ghee",Yann Kerloc'h,0,0,0,...,0,0,0,0,0,0,0,0,0,0
857,다방의 푸른 꿈,2015,[제목] 다방의 푸른 꿈\n[영문 제목] Try To Remember\n[줄거리] ...,,,"김민자, 김숙자, 김애자, Nan-yeong Lee, Kim Hae-song",Kim Dae-hyun,0,0,0,...,0,0,0,0,0,0,0,0,0,0
858,퀴어영화 나비: 어른들의 일,2015,[제목] 퀴어영화 나비: 어른들의 일\n[영문 제목] Butterfly: The A...,,,"Daeyoung Kim, Yeongdeok Lim, Gwangmu Moon, Seu...",Inkyu Baek,0,0,0,...,0,0,0,0,0,0,0,1,0,1


In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 860 entries, 0 to 859
Data columns (total 34 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   title_ko                       860 non-null    object
 1   year                           860 non-null    int64 
 2   rag_text                       860 non-null    object
 3   runtime_min                    860 non-null    object
 4   rating_kr                      860 non-null    object
 5   casts                          860 non-null    object
 6   director                       860 non-null    object
 7   genre_Action & Adventure       860 non-null    int64 
 8   genre_Reality                  860 non-null    int64 
 9   genre_SF                       860 non-null    int64 
 10  genre_Sci-Fi & Fantasy         860 non-null    int64 
 11  genre_가족                       860 non-null    int64 
 12  genre_공포                       860 non-null    int64 
 13  genre

In [20]:
import pandas as pd
from langchain_core.documents import Document
import numpy as np # NaN 값 처리를 위해 import

# 'df'는 이전 단계에서 One-Hot 인코딩까지 완료된 DataFrame이라고 가정합니다.

def df_to_langchain_documents(df: pd.DataFrame) -> list[Document]:
    """
    Pandas DataFrame을 LangChain Document 객체 리스트로 변환합니다.
    'rag_text'는 page_content로, 나머지는 metadata로 사용합니다.
    """
    documents = []
    records = df.to_dict(orient='records')
    
    for record in records:
        page_content = record.pop('rag_text', None)
        
        metadata = {}
        for key, value in record.items():
            if pd.isna(value):
                metadata[key] = None  # NaN, NaT -> None으로 변환
            else:
                metadata[key] = value
                
        # Document 객체 생성
        doc = Document(
            page_content=str(page_content), # page_content는 문자열이어야 함
            metadata=metadata           # 나머지 딕셔너리는 metadata가 됨
        )
        documents.append(doc)
        
    return documents

# --- 변환 실행 ---
try:
    documents = df_to_langchain_documents(df)

    print(f"✅ 총 {len(documents)}개의 Document 객체 생성 완료.")
    print("\n--- 첫 번째 Document 객체 샘플 ---")
    
    if documents:
        # Page Content (일부)
        print("Page Content (앞 100자):")
        print(documents[0].page_content[:100] + "...")
        
        # Metadata
        print("\nMetadata:")
        # metadata가 너무 길 수 있으니 상위 5개 키만 출력 (예시)
        metadata_sample = dict(list(documents[0].metadata.items()))
        print(metadata_sample)
        print("...")

except Exception as e:
    print(f"❌ 오류 발생: {e}")
    print("DataFrame의 'rag_text' 열이 존재하는지 확인해주세요.")

✅ 총 860개의 Document 객체 생성 완료.

--- 첫 번째 Document 객체 샘플 ---
Page Content (앞 100자):
[제목] 승부
[영문 제목] The Match
[줄거리] 세계 최고 바둑 대회에서 국내 최초 우승자가 된 조훈현. 전 국민적 영웅으로 대접받던 그는 바둑 신동이라 불리는 이창호를 ...

Metadata:
{'title_ko': '승부', 'year': 2025, 'runtime_min': 115.0, 'rating_kr': '12', 'casts': '이병헌, 유아인, 고창석, 현봉식, 문정희, 조우진, 손종학, 김강훈, 남문철, 주진모', 'director': '김형주', 'genre_Action & Adventure': 0, 'genre_Reality': 0, 'genre_SF': 0, 'genre_Sci-Fi & Fantasy': 0, 'genre_가족': 0, 'genre_공포': 0, 'genre_다큐멘터리': 0, 'genre_드라마': 1, 'genre_로맨스': 0, 'genre_모험': 0, 'genre_미스터리': 0, 'genre_범죄': 0, 'genre_스릴러': 0, 'genre_애니메이션': 0, 'genre_액션': 0, 'genre_역사': 0, 'genre_음악': 0, 'genre_전쟁': 0, 'genre_코미디': 0, 'genre_판타지': 0, 'ott_Disney Plus': 0, 'ott_FilmBox+': 0, 'ott_Netflix': 1, 'ott_Netflix Standard with Ads': 1, 'ott_TVING': 0, 'ott_Watcha': 0, 'ott_wavve': 0}
...


In [21]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

embedding = OpenAIEmbeddings(model='text-embedding-3-large')

vector_store = Chroma(
    embedding_function=embedding,
    persist_directory='./db/chromaDB2',
    collection_name='movie_rag_collection'
)
print("✅ ChromaDB 초기화 완료.")

# 500개의 Document를 하나의 배치로 설정 (안전한 크기)
BATCH_SIZE = 500
total_documents = len(documents)

# Documents 리스트를 BATCH_SIZE 단위로 분할하여 반복
for i in range(0, total_documents, BATCH_SIZE):
    batch = documents[i : i + BATCH_SIZE]
    
    # 3. ChromaDB에 배치 단위로 Documents 추가 (벡터화 및 저장 수행)
    vector_store.add_documents(documents=batch)
    
    print(f"✅ 배치 처리 완료: {i + len(batch)} / {total_documents}개 Document 저장됨.")
    
print("\n✅ 모든 Document의 ChromaDB 벡터화 및 저장 완료.")

✅ ChromaDB 초기화 완료.
✅ 배치 처리 완료: 500 / 860개 Document 저장됨.
✅ 배치 처리 완료: 860 / 860개 Document 저장됨.

✅ 모든 Document의 ChromaDB 벡터화 및 저장 완료.


In [22]:
column_names_list = df.columns.tolist()

print("\n✅ 열 이름 (List 형태):")
print(column_names_list)


✅ 열 이름 (List 형태):
['title_ko', 'year', 'rag_text', 'runtime_min', 'rating_kr', 'casts', 'director', 'genre_Action & Adventure', 'genre_Reality', 'genre_SF', 'genre_Sci-Fi & Fantasy', 'genre_가족', 'genre_공포', 'genre_다큐멘터리', 'genre_드라마', 'genre_로맨스', 'genre_모험', 'genre_미스터리', 'genre_범죄', 'genre_스릴러', 'genre_애니메이션', 'genre_액션', 'genre_역사', 'genre_음악', 'genre_전쟁', 'genre_코미디', 'genre_판타지', 'ott_Disney Plus', 'ott_FilmBox+', 'ott_Netflix', 'ott_Netflix Standard with Ads', 'ott_TVING', 'ott_Watcha', 'ott_wavve']
